In [1]:
from sqlalchemy import create_engine, text, bindparam, Integer
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [38]:
class CheckFase:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.engine = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_solmicro}@{self.server_solmicro}/{self.database_solmicro}?driver=SQL+Server')
        self.connection = None
        self.tables_info = []
        self.ruta_archivo = r'ArtIndustriConFase.xlsx'
        self.hoja_excel = 'Hoja1'

    def CargarExcel(self):
        try:
            datos_excel = pd.read_excel(
                self.ruta_archivo, sheet_name=self.hoja_excel)
            nombre_columna = 'ID'
            columna_especifica = datos_excel['Articulo']
            art_ids = set(item for item in columna_especifica)
            return art_ids
        except Exception as e:
            print("Error Excel: ", e)

    def Open_Conn_Solmicro(self):
        try:
            self.connection = self.engine.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def getDatosRuta(self, datos_excel):
        resultados = []
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in datos_excel:
                    query = text(
                        f"SELECT TOP(1) IDArticulo, Secuencia FROM tbRuta WHERE (IDArticulo = '{id}') AND  TipoOperacion=1 AND IDCentro=4500")
                    result = conn.execute(query).fetchone()
                    if not result:
                        # print(result)
                        resultados.append(id)
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            return resultados
    
    def getDatosArticulos(self, list_Arti):
        resultados = []
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in list_Arti:
                    query = text(
                        f"SELECT TOP(1) IDArticulo, DescArticulo,IDFamilia, IDTipo, IDSubfamilia FROM tbMaestroArticulo WHERE (IDArticulo = '{id}')")
                    result = conn.execute(query).fetchone()
                    if result:
                        # print(result)
                        resultados.append(result)
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            return resultados
    
    @staticmethod
    def export_to_excel(data):
        df = pd.DataFrame(data,columns=["IDArticulo","DescArticulo","IDFamilia","IDTipo","IDSubfamilia"])
        df.to_excel("ArtiSinFase.xlsx",index=False)

In [39]:
iCheck = CheckFase()
list_Art = iCheck.CargarExcel()

In [26]:
art_no_fase = iCheck.getDatosRuta(datos_excel=list_Art)

In [36]:
datos_Arti = iCheck.getDatosArticulos(list_Arti=art_no_fase)

In [40]:
iCheck.export_to_excel(data=datos_Arti)